# Who I am, agenda, etc

# What is a database index

It's a data structure used to quickly locate data in a database table without having to search through all rows. It trades space (used to store the indexes) in order to increase speed performance. A single table can have multiple indexes, each one can be associated to one or more columns of that table.

# How to create an index in Django

Index creation on Django is done when defining the model class. Django will automatically create some indexes, even if not explicitly asked to, to optimize for the most common cases.

In [ ]:
# Here we can show an example of the implicit indexes, by running sqlmigrate

# Types of indexes

# Tradeoffs

In [ ]:
# Space x Time, show examples were optimizing for the fastest speed will end up taking a lot of space in disk